**Introduction**

Intracranial hemorrhage, bleeding that occurs inside the cranium, is a serious health problem that requires rapid and often intensive medical treatment. Intracranial hemorrhages account for approximately 10% of strokes in the U.S., where stroke is the fifth-leading cause of death. Identifying the location and type of any hemorrhage present is a critical step in treating the patient.
Diagnosis requires an urgent procedure. When a patient shows acute neurological symptoms such as severe headache or loss of consciousness, highly trained specialists review medical images of the patient’s cranium to look for the presence, location and type of hemorrhage. The process is complicated and often time consuming.

For this caspstone, I am using using RSNA intracranial hemorrhage detection data set from Kaggle, (https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/overview/description) in order to identify acute intracranial hemorrhage and its subtypes from the CT study images of patients. The CT images can confirm bleeding and the evidence of trauma in head. Correct diagnosis of presence of hemorrhage and its type looking at the radiological report of the patient helps timely and effective care. The dataset consists of CT studies of patients and the probability of whether the type of hemorrhage exists or not. 


My objective is to build an algorithm that can detect hemorrhage and its type which could be a valuable information for the medical community to make data driven decisions. 


In [ ]:
#Importing the required libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import pydicom

from os import listdir

from skimage.transform import resize
from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split

from keras.applications import ResNet50
import cv2

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras import layers
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from tqdm import tqdm
import os

In [ ]:
listdir("../input/rsna-intracranial-hemorrhage-detection/")

In [ ]:
INPUT_PATH = "../input/rsna-intracranial-hemorrhage-detection/"

**Importing CSVs and Images**

The data consists of CT images as stage-1_train.zpi and stage-1_test.zip and data  for train and test. The dataset also consists of IDs of the patients as (stage_1_train.csv ) (stage_1_sample_submission.csv) and multiple labels, one for each of five sub-types of hemorrhage, plus an additional label for any, which should always be true if any of the sub-type labels is true.

I used a Kaggle kernel to avoid  donwloading this big dataset set to my local machine. Further I am using only the stage_1_train images and stage_1_train.csv  data which includes patient information of the respective images. I have splitted the training data further into train and evaluation sets.


In [ ]:
traindf = pd.read_csv(INPUT_PATH + "stage_1_train.csv")
traindf.head()

In [ ]:
traindf['type'] = traindf['ID'].str.split("_", n = 3, expand = True)[2]
traindf['PatientID'] = traindf['ID'].str.split("_", n = 3, expand = True)[1]
traindf['filename'] = traindf['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".png")
traindf.head()

In [ ]:
traindf.filename.nunique()


Let us see if the filename matches the number of images

In [ ]:
train_directory = INPUT_PATH + "stage_1_train_images/"
train_files = listdir(train_directory)

train_size = len(train_files)
train_size

We can see that the number of filenames matches the number of images in the image directory

**Data Visualization**

Let us plot a digram to visualize the distribution of label in the dataset and also see the distribution by type of hemorrhage

In [ ]:
traindf['Label'].value_counts()
sns.countplot(x= 'Label', data = traindf);

We can see the data imbalance here (The images with hemorrhage being less). We will try balancing before building our model

In [ ]:
traindf['Label'].value_counts()
sns.countplot(x= 'Label', data = traindf, hue = 'type');

**Preprocessing the images**

Rescaling resizing and converting the .dcm imgaes to .png

In [ ]:
def window_image(img, window_center,window_width, intercept, slope, rescale=True):

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    
    if rescale:
        # Extra rescaling to 0-1, not in the original notebook
        img = (img - img_min) / (img_max - img_min)
    
    return img
    
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]


def view_images(images, title = '', aug = None):
    width = 5
    height = 2
    fig, axs = plt.subplots(height, width, figsize=(15,5))
    
    for im in range(0, height * width):
        data = pydicom.read_file(os.path.join(INPUT_PATH + "stage_1_train_images/",'ID_'+images[im]+ '.dcm'))
        image = data.pixel_array
        window_center , window_width, intercept, slope = get_windowing(data)
        image_windowed = window_image(image, window_center, window_width, intercept, slope)


        i = im // width
        j = im % width
        axs[i,j].imshow(image_windowed, cmap=plt.cm.bone) 
        axs[i,j].axis('off')
        
    plt.suptitle(title)
    plt.show()

In [ ]:
def save_and_resize(filenames, load_dir):   
    X=[] 
    
    save_dir = '/kaggle/tmp/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for filename in tqdm(filenames):
        path = load_dir + filename
        new_path = save_dir + filename.replace( '.dcm', '.png' )
        
        dcm = pydicom.dcmread(path)
        window_center , window_width, intercept, slope = get_windowing(dcm)
        img = dcm.pixel_array
        img = window_image(img, window_center, window_width, intercept, slope)
        
        resized = cv2.resize(img, (224, 224)) 
        res = cv2.imwrite(new_path, resized)
        if not res:
            print('Failed')

**Visualization of images**

In [ ]:
train_files[0:5]

In [ ]:
#Lets see what the images contain
dataset = pydicom.dcmread(train_directory + "ID_3b59681d3.dcm")
print(dataset)

In [ ]:
subtypes = traindf.type.unique()
subtypes

In [ ]:
view_images(traindf[(traindf['type'] == 'epidural') & (traindf['Label'] == 1)][:10].PatientID.values, title = 'Images of Epidural Hemorrhage')

In [ ]:
view_images(traindf[(traindf['type'] == 'intraparenchymal') & (traindf['Label'] == 1)][:10].PatientID.values, title = 'Images of Intraparenchymal Hemorrhage')

In [ ]:
view_images(traindf[(traindf['type'] == 'intraventricular') & (traindf['Label'] == 1)][:10].PatientID.values, title = 'Images of Intraventricular Hemorrhage')

In [ ]:
view_images(traindf[(traindf['type'] == 'subarachnoid') & (traindf['Label'] == 1)][:10].PatientID.values, title = 'Images of Subarachnoid Hemorrhage')

In [ ]:
view_images(traindf[(traindf['type'] == 'subdural') & (traindf['Label'] == 1)][:10].PatientID.values, title = 'Images of Subdural Hemorrhage')

**Preparing the dataframe
**



In [ ]:
traindf.head(2)

We are taking random sample of 1000 images. This is only for faster run time, with more CPU, GPU we can use all images. 

In [ ]:
np.random.seed(0)
sample_data = np.random.choice((train_files), 1000)  
sample_df = traindf[traindf.filename.apply(lambda x: x.replace('.png', '.dcm')).isin(sample_data)]

pivot_df = sample_df[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()
print(pivot_df.shape)
pivot_df.head(2)

In [ ]:
import os
save_and_resize(filenames=sample_data, load_dir=INPUT_PATH + "stage_1_train_images/")

Reading each of the 1000 images with opencv and saving as matrix 

In [ ]:
X=[]
y=[] 
path =  '/kaggle/tmp/' 
import cv2  
import glob 

for i, row in pivot_df.iterrows():  
    file = row['filename'] 
    if path+ file in glob.glob('/kaggle/tmp/*.png') : 
        img=path+file
        img=cv2.imread(img, 0) #grey 
        #print(img.shape)
        X.append(img)
        y.append(row['any'])  
    else: 
        continue

In [ ]:
X= np.array(X)
y= np.array(y)

In [ ]:
X.shape, y.shape

In [ ]:
plt.hist(y);

We can see the data is imbalanced. We will balance the data by undersampling the majority class (this is to save the omputational time as well due to the limitation of CPU/GPU)

Let us split the data to train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Balancing the data by undersample the majority class

In [ ]:
#Oversample
#from imblearn.over_sampling import SMOTE
#Let us upsample the dataset
#oversampler= SMOTE(random_state=0)
#X_train_final, y_train_final = oversampler.fit_sample(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

In [ ]:
# print(X_train_final.shape)
# print(y_train_final.shape)

In [ ]:
#Undersample
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_train_final, y_train_final = rus.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

In [ ]:
X_train_final.shape, y_train_final.shape

Normalizing the pixel values between 0 and 1

In [ ]:
X_train= X_train_final.reshape(X_train_final.shape[0], 224, 224)/255
y_train = y_train_final 
X_test = X_test/255
y_test = y_test

In [ ]:
X_train= X_train/255
y_train = y_train
X_test = X_test/255
y_test = y_test

In [ ]:
img_rows, img_cols = 224, 224
X_train = X_train.reshape((-1, img_rows, img_cols, 1))
y_train = y_train
X_test = X_test.reshape((-1, img_rows, img_cols, 1))

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
plt.hist(y_train);

We can see the balanced train data set

In [ ]:
plt.imshow(X_train[1].reshape(224, 224)) 
print(y_train[1])

**Building Models**

Dense model

In [ ]:
from keras.layers import Add
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 


 
    
# class_weight = {0: 10,
#                 1: 1 } 

model_dense = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model_dense.add(Dense(32, activation='relu', input_shape=(224, 224,1)))
# Dropout layers remove features and fight overfitting
model_dense.add(Dropout(0.4))
model_dense.add(Dense(32, activation='relu'))
model_dense.add(Dropout(0.4))
# End with a number of units equal to the number of classes we have for our outcome
model_dense.add(Flatten())
model_dense.add(Dense(1, activation='softmax'))


model_dense.summary()


# Compile the model to put it all together.
model_dense.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history_dense = model_dense.fit(X_train, y_train,
                          batch_size=64,
                          epochs=3,
                          verbose=1,
                          validation_data=(X_test, y_test))
score = model_dense.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_pred= model_dense.predict(X_test)

In [ ]:
#Model Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) 

Convolutional Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 


batch_size=64
validation_ratio=0.1
sample_size=200
epochs=3 

model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(224, 224,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())  

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train,y_train,batch_size=batch_size,epochs=3,validation_data=(X_test, y_test))


In [ ]:
y_pred= model.predict(X_test)

In [ ]:
#Model Evaluation
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

### ResNet

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# Reshape channel to 3 for resnet
# One hot on y 
from keras.utils.np_utils import to_categorical
X_train_rgb = np.repeat(X_train, 3, -1)
X_test_rgb = np.repeat(X_test, 3, -1)
y_train_rgb= to_categorical(y_train)
y_test_rgb= to_categorical(y_test)
X_train_rgb.shape , X_test_rgb.shape, y_train_rgb.shape, y_test_rgb.shape

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50 
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, BatchNormalization, Dropout , Concatenate
from keras import backend as K

        
base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
last_layer = base_model.output
x = GlobalAveragePooling2D()(last_layer)
x = Dense(6, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
out = Dense(2, activation='softmax',name='output_layer')(x)


resnet_model = Model(inputs=base_model.input, outputs=out)
resnet_model.summary()

for layer in resnet_model.layers[:-4]:
    layer.trainable = False

resnet_model.layers[-1].trainable
resnet_model.compile(loss='categorical_crossentropy',
                                 optimizer='adam',metrics=['accuracy'])
resnet_model.fit(X_train_rgb, y_train_rgb, batch_size=32, epochs=3, verbose=1, validation_data=(X_test_rgb, y_test_rgb))

In [ ]:
y_pred = resnet_model.predict(X_test_rgb)

In [ ]:
y_pred=np.argmax(y_pred, 1)

In [ ]:
#Model Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) 

**VGG Model**

In [ ]:
from keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
last_layer = base_model.output
x = GlobalAveragePooling2D()(last_layer)
x = Dense(6, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
out = Dense(2, activation='softmax',name='output_layer')(x)


vgg_model = Model(inputs=base_model.input, outputs=out)
vgg_model.summary()

# for layer in resnet_model.layers[:-4]:
#     layer.trainable = False

vgg_model.layers[-1].trainable
vgg_model.compile(loss='categorical_crossentropy',
                                 optimizer='adam',metrics=['accuracy'])
vgg_model.fit(X_train_rgb, y_train_rgb, batch_size=32, epochs=5, verbose=1, validation_data=(X_test_rgb, y_test_rgb))

In [ ]:
y_pred = vgg_model.predict(X_test_rgb)
y_pred=np.argmax(y_pred, 1)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) 

Conclusions

We have created workable pipelines to preprocess dcm data and fit into severeal CNNs 

The models haven't led to useable results due to the limitation on computational power. 





Next steps

Train models on AWS/google cloud using more images (with augumentation) as input 

Try fastai and other pretrained models

Compare model performance

Create front end tool/app for users to interact with the model results 


